# Analysis-Ready, Cloud Optimized (ARCO) ERA 5 Data and _You_


Google Research & Cloud are in the process of converting a large corpus of ERA 5 data from grib into an [ARCO dataset](https://www.frontiersin.org/articles/10.3389/fclim.2021.782909/full), namely in the [Zarr format](https://zarr.readthedocs.io/). This notebook demonstrates how to access currently available datasets and make them useful. 

To hear about the current status of ERA 5 preparation, please check out [our roadmap](https://github.com/google-research/arco-era5#roadmap).

For now, let's start working with the data!

> *Pre-requisites*: Make sure you install the all notebook requirements, defined [here](https://github.com/google-research/arco-era5/tree/main/docs/environment.yml):
> ```
> conda env create -f environment.yml
> conda activate era5-docs
> ```

## Opening the Data

So far, the ERA 5 dataset has been cloud optimized; it's not fully analysis-ready yet. However, that doesn't mean that it's not immediately useful!

Let's open the Zarr data and take a look at it. We can do this with [XArray](https://xarray.pydata.org/en/stable/), once we get the right GCS permissions. We can test bucket access with [fsspec](https://filesystem-spec.readthedocs.io/en/latest/index.html):


In [1]:
import fsspec

fs = fsspec.filesystem('gs')
fs.ls('gs://weatherbench2/datasets/era5/')

['weatherbench2/datasets/era5/',
 'weatherbench2/datasets/era5/1959-2022-1h-240x121_equiangular_with_poles_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-1h-360x181_equiangular_with_poles_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-128x64_equiangular_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-128x64_equiangular_with_poles_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-240x121_equiangular_with_poles_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-512x256_equiangular_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_with_poles_conservative.zarr',
 'weatherbench2/datasets/era5/1959-2022-6h-64x33.zarr',
 'weatherbench2/datasets/era5/1959-2022-full_37-1h-0p25deg-chunk-1.zarr-v2',
 'weatherbench2/datasets/era5/1959-2022-full_37-6h-0p25deg-chu

Next, we'll load the 6-hour downsampled data set (including derived variables)

In [22]:
import xarray as xr

reanalysis = xr.open_zarr(
    'gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr', 
    chunks={'time': 48},
    consolidated=True,
    decode_timedelta=True    
)

We can slice the data across time using [standard XArray methods](https://xarray.pydata.org/en/stable/user-guide/indexing.html). For example:

In [ ]:
recent_an = reanalysis.sel(time=slice('2020-01-01', '2021-01-01'))
geopotential_t0 = recent_an.z[0, :]
most_recent_ds = recent_an.isel(time=-1)
new_years_ds = recent_an.sel(time='2020-01-01')

Let's reduce the size by only including data since 1/1/2014 and only our target variables:
- 10m_wind_speed
- 2m_temperature
- high_vegetation_cover
- low_vegetation_cover
- lake_cover
- leaf_area_index_high_vegetation
- leaf_area_index_low_vegetation
- snow_depth
- temperature
- total_precipitation_12hr
- total_precipitation_24hr
- total_precipitation_6hr
- type_of_high_vegetation
- type_of_low_vegetation
- wind_speed
- vorticity

In [23]:
#Select time values since 1/1/2014 and the following variables: latitude, longitude, time, 10m_wind_speed, 2m_temperature, high_vegetation_cover, low_vegetation_cover, lake_cover, leaf_area_index_high_vegetation, leaf_area_index_low_vegetation, snow_depth, total_precipitation_12hr, total_precipitation_24hr, total_precipitation_6hr, type_of_high_vegetation, type_of_low_vegetation
reanalysis = reanalysis.sel(time=slice('2014', '2023'))[['latitude', 'longitude', 'time', '10m_wind_speed', '2m_temperature', 'high_vegetation_cover', 'low_vegetation_cover', 'lake_cover', 'leaf_area_index_high_vegetation', 'leaf_area_index_low_vegetation', 'snow_depth', 'total_precipitation_12hr', 'total_precipitation_24hr', 'total_precipitation_6hr', 'type_of_high_vegetation', 'type_of_low_vegetation']]

Next, we'll restrict our data to (just) the continental US, which has longitude boundaries of -125 and -66.96; and latitude boundaries of 24.5 and 49.

Note that the [coordinate system for ERA 5](https://confluence.ecmwf.int/display/CKB/ERA5%3A+What+is+the+spatial+reference#ERA5:Whatisthespatialreference-Coordinatesystem) is [-90, +90] for latitude and [0, 360] for longitude with respect to the Greenwich Prime Meridian (i.e. different from normal [-180, 180]). We can include an auxiliary function to [assign new coordinates](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.assign_coords.html) to let us use ``standard" coordinates.

In [28]:
def lon_to_360(dlon: float) -> float:
  return ((360 + (dlon % 360)) % 360)


US_ds = reanalysis.where(
    (reanalysis.longitude > lon_to_360(-125)).compute() & (reanalysis.latitude > 24.5).compute() &
    (reanalysis.longitude < lon_to_360(-66.96)).compute() & (reanalysis.latitude < 49).compute(),
    drop=True
)
US_ds

<xarray.Dataset> Size: 9GB
Dimensions:                          (time: 13188, latitude: 97, longitude: 232)
Coordinates:
  * latitude                         (latitude) float32 388B 48.75 ... 24.75
  * longitude                        (longitude) float32 928B 235.2 ... 293.0
  * time                             (time) datetime64[ns] 106kB 2014-01-01 ....
Data variables: (12/13)
    10m_wind_speed                   (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    2m_temperature                   (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    high_vegetation_cover            (latitude, longitude) float32 90kB dask.array<chunksize=(97, 232), meta=np.ndarray>
    low_vegetation_cover             (latitude, longitude) float32 90kB dask.array<chunksize=(97, 232), meta=np.ndarray>
    lake_cover                       (latitude, longitude) float32 90kB dask.array<chunksize=(97, 232), meta=np.ndarray>
    leaf_area_index_high_vegetation  (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    ...                               ...
    snow_depth                       (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    total_precipitation_12hr         (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    total_precipitation_24hr         (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    total_precipitation_6hr          (time, latitude, longitude) float32 1GB dask.array<chunksize=(44, 97, 232), meta=np.ndarray>
    type_of_high_vegetation          (latitude, longitude) float32 90kB dask.array<chunksize=(97, 232), meta=np.ndarray>
    type_of_low_vegetation           (latitude, longitude) float32 90kB dask.array<chunksize=(97, 232), meta=np.ndarray>

We can try saving the zarr file locally

In [ ]:
#Adjust the chunks of US_ds to be uniform in size except for the last one
US_ds = US_ds.chunk({'time': 48})

#Export US_ds as a local zarr file using zstd codec for all variables

from zarr.codecs import BloscCodec

compressor = BloscCodec(cname="zstd", clevel=3, shuffle="shuffle")

US_ds.to_zarr('US_ds.zarr', mode='w', encoding={v: {'compressor': compressor} for v in US_ds.data_vars})

/opt/homebrew/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/zarr/core/group.py:2471: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/opt/homebrew/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/zarr/core/group.py:2471: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/opt/homebrew/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/zarr/core/group.py:2471: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/opt/homebrew/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/zarr/core/group.py:2471: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(
/opt/homebrew/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/zarr/core/gr

TypeError: Expected a BytesBytesCodec. Got <class 'numcodecs.blosc.Blosc'> instead.

We can check out the size of this final dataset. It appears to be roughly 9.5 Gigabytes--much more manageable than the full dataset (which is 80 TiB), but still probably too large to load into memory.

In [13]:
# Compute the size of the US_ds dataset
print(f'size: {US_ds.nbytes / (1024 ** 4)} TiB')

size: 0.008638021394290263 TiB


Let's try taking a single observation and converting it to a dataframe

In [40]:
ds_single = US_ds.sel(time=slice("2014-01-01T00","2014-01-01T06"))


In [41]:
#Convert ds_single to a dataframe
df = ds_single.to_dataframe()

In [42]:
df.head(20)

10m_wind_speed  2m_temperature  \
time       latitude longitude                                   
2014-01-01 48.75    235.25           1.079360      280.156342   
                    235.50           0.822210      279.047455   
                    235.75           0.290597      278.663147   
                    236.00           0.436304      278.727722   
                    236.25           0.879576      280.109100   
                    236.50           1.260939      280.786407   
                    236.75           1.557131      280.498138   
                    237.00           1.398971      280.490265   
                    237.25           1.220213      280.414673   
                    237.50           0.856412      279.847626   
                    237.75           0.557090      277.921265   
                    238.00           0.270446      275.153778   
                    238.25           0.670179      274.194519   
                    238.50           1.037463      273.591248   
                    238.75           1.232416      273.504608   
                    239.00           1.366004      272.483948   
                    239.25           1.432762      271.274261   
                    239.50           1.292196      269.880280   
                    239.75           1.386616      269.853485   
                    240.00           1.611844      270.538666   

                               high_vegetation_cover  low_vegetation_cover  \
time       latitude longitude                                                
2014-01-01 48.75    235.25                  0.959074              0.009064   
                    235.50                  0.986694              0.010254   
                    235.75                  0.986694              0.010254   
                    236.00                  0.991668              0.005341   
                    236.25                  0.695039              0.002686   
                    236.50                  0.695039              0.002686   
                    236.75                  0.000000              0.000000   
                    237.00                  0.000000              0.000000   
                    237.25                  0.000000              0.000000   
                    237.50                  0.903621              0.018388   
                    237.75                  0.903621              0.018388   
                    238.00                  0.927395              0.050784   
                    238.25                  0.849389              0.129233   
                    238.50                  0.849389              0.129233   
                    238.75                  0.911373              0.088627   
                    239.00                  0.989196              0.010804   
                    239.25                  0.989196              0.010804   
                    239.50                  0.941098              0.058902   
                    239.75                  0.941098              0.058902   
                    240.00                  0.986587              0.013413   

                               lake_cover  leaf_area_index_high_vegetation  \
time       latitude longitude                                                
2014-01-01 48.75    235.25       0.125433                              NaN   
                    235.50       0.040605                              NaN   
                    235.75       0.019502                              NaN   
                    236.00       0.021318                              NaN   
                    236.25       0.150947                              NaN   
                    236.50       0.166267                              NaN   
                    236.75       0.027711                              NaN   
                    237.00       0.000000                              NaN   
                    237.25       0.019700                              NaN   
                    237.5

In [6]:
import metview as mv

# Convert a slice of the data to grib on your local filesystem.
ds_single = US_ds.sel(time=slice("2014-01-01T00","2014-01-01T06"))
single_fieldset = mv.dataset_to_fieldset(ds_single.squeeze())

/opt/homebrew/anaconda3/envs/erdos_spring_2025/lib/python3.12/site-packages/cfgrib/xarray_to_grib.py:252: FutureWarning: GRIB write support is experimental, DO NOT RELY ON IT!
  warnings.warn("GRIB write support is experimental, DO NOT RELY ON IT!", FutureWarning)


Error trying to write xarray dataset to GRIB for conversion to Metview Fieldset


KeyboardInterrupt: 

In [45]:
single_fieldset.describe()

parameter,typeOfLevel,level,date,time,step,number,paramId,class,stream,type,experimentVersionNumber
t,surface,0,"20070323,20140101","0,600,...",0,None,130,None,None,None,None


In [4]:
# Compute other derived variables, such as wind speed
u_wind = single_fieldset.select(shortName='100u')
v_wind = single_fieldset.select(shortName='100v')
speed100 = mv.speed(u_wind,v_wind)
u_wind = single_fieldset.select(shortName='10u')
v_wind = single_fieldset.select(shortName='10v')
speed10 = mv.speed(u_wind,v_wind)
speed = mv.merge(speed10,speed100)
single_fieldset = mv.merge(single_fieldset, speed)

In [48]:
def roll_longitude(ds):
    return ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180)).sortby('longitude')

In [49]:
# Regrid our dataset to lat/long coordinates (at 0.25° x 0.25°).
single_ll = mv.read(data=single_fieldset, grid=[0.25,0.25])

ds_single_ll = single_ll.to_dataset()
ds_single_ll = roll_longitude(ds_single_ll)
ds_single_ll

: 

## Preview: Other cloud-optimized datasets

Beyond the surface level reanalysis, we've also made a few other datasets available. You can see previews of each of these in the next section. Stay tuned – we're actively developing demo notebooks.

### Surface forecasts

In [17]:
import xarray as xr

forecasts = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/co/single-level-forecast.zarr/', 
    chunks={'time': 48},
    consolidated=True,
)

In [18]:
print(f'size: {forecasts.nbytes / (1024 ** 4)} TiB')

size: 24.524807091955154 TiB


In [19]:
forecasts

<xarray.Dataset>
Dimensions:     (time: 31168, step: 19, values: 542080)
Coordinates:
    latitude    (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude   (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    number      int64 ...
  * step        (step) timedelta64[ns] 00:00:00 01:00:00 ... 17:00:00 18:00:00
    surface     float64 ...
  * time        (time) datetime64[ns] 1979-01-01T06:00:00 ... 2021-08-31T18:0...
    valid_time  (time, step) datetime64[ns] dask.array<chunksize=(48, 19), meta=np.ndarray>
Dimensions without coordinates: values
Data variables: (12/21)
    cp          (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    crr         (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    csf         (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    csfr        (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    es          (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    gwd         (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    ...          ...
    smlt        (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    ssrd        (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    str         (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    strd        (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    tp          (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
    ttr         (time, step, values) float32 dask.array<chunksize=(48, 19, 542080), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              1
    GRIB_subCentre:            0
    history:                   2022-09-27T00:23 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  011ff1003a2ecdbcbd5c5ecc9c0a2b53a2a3056863d691...
    pangeo-forge:recipe_hash:  79422b41ca015cbc01df0929ac7bea8c7927f70e2c1e53...
    pangeo-forge:version:      0.9.1

### Atmospheric data: Model level data

See the [Model Levels Walkthrough](1-Model-Levels-Walkthrough.ipynb) for an in-depth look at these datasets...


In [22]:
import xarray as xr

ml_moisture = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/co/model-level-moisture.zarr', 
    chunks={'time': 48},
    consolidated=True,
)

In [23]:
print(f'{ml_moisture.nbytes / (1024 ** 4)} TiB')

707.3467227025685 TiB


In [24]:
ml_moisture

<xarray.Dataset>
Dimensions:     (time: 374016, hybrid: 137, values: 542080)
Coordinates:
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
    latitude    (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    longitude   (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
    step        timedelta64[ns] ...
  * time        (time) datetime64[ns] 1979-01-01 ... 2021-08-31T23:00:00
    valid_time  (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    cc          (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    ciwc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    clwc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    crwc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    cswc        (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    o3          (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
    q           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 542080), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-09-27T00:00 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  06482fd143803f9b1ae4d85151c399e57173179ee57bcf...
    pangeo-forge:recipe_hash:  7af5b330ea42d563af5c8b6d2606cb839306f90d83bfd2...
    pangeo-forge:version:      0.9.1

In [25]:
import xarray as xr

ml_wind = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/co/model-level-wind.zarr', 
    chunks={'time': 48},
    consolidated=True,
)

In [26]:
print(f'{ml_wind.nbytes / (1024 ** 4)} TiB')

305.8925611573068 TiB


In [27]:
ml_wind

<xarray.Dataset>
Dimensions:     (time: 374016, hybrid: 137, values: 410240)
Coordinates:
  * hybrid      (hybrid) float64 1.0 2.0 3.0 4.0 5.0 ... 134.0 135.0 136.0 137.0
    step        timedelta64[ns] ...
  * time        (time) datetime64[ns] 1979-01-01 ... 2021-08-31T23:00:00
    valid_time  (time) datetime64[ns] dask.array<chunksize=(48,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    d           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
    t           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
    vo          (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
    w           (time, hybrid, values) float32 dask.array<chunksize=(48, 137, 410240), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    European Centre for Medium-Range Weather Forec...
    GRIB_edition:              2
    GRIB_subCentre:            0
    history:                   2022-09-10T06:30 GRIB to CDM+CF via cfgrib-0.9...
    institution:               European Centre for Medium-Range Weather Forec...
    pangeo-forge:inputs_hash:  6a55b85e7eec10e5e6f4ae6f3f7f4ea0160471c527d9a1...
    pangeo-forge:recipe_hash:  0d8fbfdcac5d1f6bafdd1606da47c339d37dbdd7c8f2ba...
    pangeo-forge:version:      0.9.1